In [2]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 17360, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 17360 (delta 36), reused 18 (delta 18), pack-reused 17308 (from 2)
Receiving objects: 100% (17360/17360), 16.23 MiB | 27.39 MiB/s, done.
Resolving deltas: 100% (11898/11898), done.


In [2]:
import os
os.chdir('yolov5')

In [3]:
ls

benchmarks.py    data/       LICENSE         README.zh-CN.md   tutorial.ipynb
CITATION.cff     detect.py   models/         requirements.txt  utils/
classify/        export.py   pyproject.toml  segment/          val.py
CONTRIBUTING.md  hubconf.py  README.md       train.py


In [4]:
os.mkdir('data_images')

In [5]:
train= 'data_images/train'
test = 'data_images/test'
os.mkdir(train)
os.mkdir(test)

In [6]:
ls

benchmarks.py    data/         hubconf.py      README.md         train.py
CITATION.cff     data_images/  LICENSE         README.zh-CN.md   tutorial.ipynb
classify/        detect.py     models/         requirements.txt  utils/
CONTRIBUTING.md  export.py     pyproject.toml  segment/          val.py


In [7]:
os.chdir('/kaggle/input')

In [8]:
import os
from glob import glob
import pandas as pd
from functools import reduce
from xml.etree import ElementTree as et

In [9]:
xml_list=glob("/kaggle/input/pascal-voc-2012-dataset/VOC2012_train_val/VOC2012_train_val/Annotations/*.xml")

In [ ]:
#xml_list

In [10]:
#read xml files
# extract filename,size(width,height),object(name,xmin,xmax,ymin,ymax)
def extract_text(filename):
    tree=et.parse(filename)
    root=tree.getroot()
    
    #extract file name
    image_name=root.find('filename').text
    
    #width and height of the image
    width=root.find('size').find('width').text
    height=root.find('size').find('height').text
    
    objs =root.findall('object')
    parser =[]
    for obj in objs:
        name=obj.find('name').text
        bndbox=obj.find('bndbox')
        xmin=bndbox.find('xmin').text
        xmax=bndbox.find('xmax').text
        ymin=bndbox.find('ymin').text
        ymax=bndbox.find('ymax').text
        parser.extend([image_name,width,height,name,xmin,xmax,ymin,ymax])
    return parser

In [11]:
parser_all=list(map(extract_text,xml_list))

In [12]:
data_train = reduce(lambda x,y : x+y,parser_all)

In [13]:
dataList = []
i=0
while(i<len(data_train)):
    dataList.append(data_train[i:i+8])
    i+=8

In [ ]:
#dataList

In [14]:
df = pd.DataFrame(dataList,columns=['filename','width','height','name','xmin','xmax','ymin','ymax'])
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax
0,2011_006424.jpg,333,500,person,67,269,56,369
1,2010_003717.jpg,500,375,sofa,34,498,1,242
2,2010_003717.jpg,500,375,bottle,432,500,305,332
3,2009_000488.jpg,500,375,tvmonitor,21,193,1,95
4,2009_000488.jpg,500,375,cat,224,287,159,229


In [15]:
# Convert numeric columns safely (round float values before conversion)
numeric_columns = ['width', 'height', 'xmin', 'xmax', 'ymin', 'ymax']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce').round().astype('int')

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40138 entries, 0 to 40137
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  40138 non-null  object
 1   width     40138 non-null  int64 
 2   height    40138 non-null  int64 
 3   name      40138 non-null  object
 4   xmin      40138 non-null  int64 
 5   xmax      40138 non-null  int64 
 6   ymin      40138 non-null  int64 
 7   ymax      40138 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 2.4+ MB


In [17]:
#center x,center y
df['center_x']=((df['xmax']+df['xmin'])/2)/df['width']
df['center_y']=((df['ymax']+df['ymin'])/2)/df['height']
df['w']=(df['xmax']-df['xmin'])/df['width']
df['h']=(df['ymax']-df['ymin'])/df['height']


In [18]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,2011_006424.jpg,333,500,person,67,269,56,369,0.504505,0.425000,0.606607,0.626000
1,2010_003717.jpg,500,375,sofa,34,498,1,242,0.532000,0.324000,0.928000,0.642667
2,2010_003717.jpg,500,375,bottle,432,500,305,332,0.932000,0.849333,0.136000,0.072000
3,2009_000488.jpg,500,375,tvmonitor,21,193,1,95,0.214000,0.128000,0.344000,0.250667
4,2009_000488.jpg,500,375,cat,224,287,159,229,0.511000,0.517333,0.126000,0.186667


In [19]:
images=df['filename'].unique()

In [20]:
len(images)

17125

In [21]:
 #test data set
xml_test=glob("/kaggle/input/pascal-voc-2012-dataset/VOC2012_test/VOC2012_test/Annotations/*.xml")


In [22]:
parser_all_test=list(map(extract_text,xml_test))

In [23]:
data_test = reduce(lambda x,y : x+y,parser_all_test)

In [24]:
dataListTest = []
i=0
while(i<len(data_test)):
    dataListTest.append(data_test[i:i+8])
    i+=8

In [25]:
# dataListTest
df_test = pd.DataFrame(dataListTest,columns=['filename','width','height','name','xmin','xmax','ymin','ymax'])
df_test.head()

,filename,width,height,name,xmin,xmax,ymin,ymax
0,2010_006298.jpg,500,375,person,244,285,135,200
1,2010_006298.jpg,500,375,person,251,319,139,247
2,2012_003858.jpg,336,448,person,29,161,176,448
3,2012_003788.jpg,500,394,person,257,458,11,331
4,2011_006625.jpg,375,500,person,115,320,1,367


In [26]:
# Convert numeric columns safely (round float values before conversion)
numeric_columns = ['width', 'height', 'xmin', 'xmax', 'ymin', 'ymax']
df_test[numeric_columns] = df_test[numeric_columns].apply(pd.to_numeric, errors='coerce').round().astype('int')

In [27]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7330 entries, 0 to 7329
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  7330 non-null   object
 1   width     7330 non-null   int64 
 2   height    7330 non-null   int64 
 3   name      7330 non-null   object
 4   xmin      7330 non-null   int64 
 5   xmax      7330 non-null   int64 
 6   ymin      7330 non-null   int64 
 7   ymax      7330 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 458.2+ KB


In [28]:
#center x,center y
df_test['center_x']=((df_test['xmax']+df_test['xmin'])/2)/df_test['width']
df_test['center_y']=((df_test['ymax']+df_test['ymin'])/2)/df_test['height']
df_test['w']=(df_test['xmax']-df_test['xmin'])/df_test['width']
df_test['h']=(df_test['ymax']-df_test['ymin'])/df_test['height']

In [29]:
df_test.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,2010_006298.jpg,500,375,person,244,285,135,200,0.529000,0.446667,0.082000,0.173333
1,2010_006298.jpg,500,375,person,251,319,139,247,0.570000,0.514667,0.136000,0.288000
2,2012_003858.jpg,336,448,person,29,161,176,448,0.282738,0.696429,0.392857,0.607143
3,2012_003788.jpg,500,394,person,257,458,11,331,0.715000,0.434010,0.402000,0.812183
4,2011_006625.jpg,375,500,person,115,320,1,367,0.580000,0.368000,0.546667,0.732000


In [30]:
img_train = tuple(df['filename'].unique())
img_test = tuple(df_test['filename'].unique())

In [31]:
len(img_train),len(img_test)

(17125, 5138)

In [32]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,2011_006424.jpg,333,500,person,67,269,56,369,0.504505,0.425000,0.606607,0.626000
1,2010_003717.jpg,500,375,sofa,34,498,1,242,0.532000,0.324000,0.928000,0.642667
2,2010_003717.jpg,500,375,bottle,432,500,305,332,0.932000,0.849333,0.136000,0.072000
3,2009_000488.jpg,500,375,tvmonitor,21,193,1,95,0.214000,0.128000,0.344000,0.250667
4,2009_000488.jpg,500,375,cat,224,287,159,229,0.511000,0.517333,0.126000,0.186667


In [33]:
df_test.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,2010_006298.jpg,500,375,person,244,285,135,200,0.529000,0.446667,0.082000,0.173333
1,2010_006298.jpg,500,375,person,251,319,139,247,0.570000,0.514667,0.136000,0.288000
2,2012_003858.jpg,336,448,person,29,161,176,448,0.282738,0.696429,0.392857,0.607143
3,2012_003788.jpg,500,394,person,257,458,11,331,0.715000,0.434010,0.402000,0.812183
4,2011_006625.jpg,375,500,person,115,320,1,367,0.580000,0.368000,0.546667,0.732000


In [34]:
#Label encoding
# df['name'].value_counts()
def label_encoding(x):
        labels = {'person':0,'car':1,'chair':2,'bottle':3,'pottedplant':4,'bird':5,
                  'dog':6,'sofa':7,'bicycle':8,'horse':9,'boat':10,'motorbike':11,'cat':12,
                  'tvmonitor':13,'cow':14,'sheep':15,'aeroplane':16,'train':17,'diningtable':18,
                  'bus':19}
        return labels[x]

In [35]:
df_train=df

In [36]:
df_train['id']=df_train['name'].apply(label_encoding)
df_test['id']=df_test['name'].apply(label_encoding)

In [37]:
df_train['id'].unique(),df_test['id'].unique()

(array([ 0,  7,  3, 13, 12,  4,  9,  1,  6, 17,  8, 16, 18, 11,  2, 14, 19,
         5, 10, 15]),
 array([ 0,  2, 18]))

In [38]:
import os
from shutil import copy


In [39]:
cols = ['filename','id','center_x','center_y','w','h']
group_obj_train=df_train[cols].groupby('filename')
group_obj_test=df_test[cols].groupby('filename')

In [40]:
#save each image in train/test folder and respective labels in  .txt
def save_data(filename,folder_path,group_obj):
    src = os.path.join('/kaggle/input/pascal-voc-2012-dataset/VOC2012_train_val/VOC2012_train_val/JPEGImages',filename)
    dst = os.path.join(folder_path,filename)
    copy(src,dst)

    #save the labels
    text_filename=os.path.join(folder_path,os.path.splitext(filename)[0]+'.txt') 
    group_obj.get_group(filename).set_index('filename').to_csv(text_filename,sep=' ',index=False,header=False)
              

In [41]:
filename_series= pd.Series(group_obj_train.groups.keys())

In [42]:
filename_series

0        2007_000027.jpg
1        2007_000032.jpg
2        2007_000033.jpg
3        2007_000039.jpg
4        2007_000042.jpg
              ...       
17120    2012_004326.jpg
17121    2012_004328.jpg
17122    2012_004329.jpg
17123    2012_004330.jpg
17124    2012_004331.jpg
Length: 17125, dtype: object

In [43]:
filename_series.apply(save_data,args=('/kaggle/working/yolov5/data_images/train',group_obj_train))

0        None
1        None
2        None
3        None
4        None
         ... 
17120    None
17121    None
17122    None
17123    None
17124    None
Length: 17125, dtype: object

In [44]:
filename_series_test= pd.Series(group_obj_test.groups.keys())

In [45]:
filename_series_test

0       2008_000200.jpg
1       2008_000210.jpg
2       2008_000216.jpg
3       2008_000228.jpg
4       2008_000263.jpg
             ...       
5133    2012_004322.jpg
5134    2012_004323.jpg
5135    2012_004324.jpg
5136    2012_004325.jpg
5137    2012_004327.jpg
Length: 5138, dtype: object

In [46]:
#save each image in train/test folder and respective labels in  .txt
def save_data(filename,folder_path,group_obj):
    src = os.path.join('/kaggle/input/pascal-voc-2012-dataset/VOC2012_test/VOC2012_test/JPEGImages',filename)
    dst = os.path.join(folder_path,filename)
    copy(src,dst)

    #save the labels
    text_filename=os.path.join(folder_path,os.path.splitext(filename)[0]+'.txt') 
    group_obj.get_group(filename).set_index('filename').to_csv(text_filename,sep=' ',index=False,header=False)
              

In [47]:
filename_series_test.apply(save_data,args=('/kaggle/working/yolov5/data_images/test',group_obj_test))

0       None
1       None
2       None
3       None
4       None
        ... 
5133    None
5134    None
5135    None
5136    None
5137    None
Length: 5138, dtype: object

In [48]:
import os
os.chdir("/kaggle/working/yolov5")

In [49]:
# Define file name and content
filename = "/kaggle/working/yolov5/data.yaml"  # Save in /kaggle/working/
content = """
train:  data_images/train
val: data_images/test
nc: 19
names: ['person',
        'car',
        'chair',
        'bottle',
        'pottedplant',
        'bird',
        'dog',
        'sofa',
        'bicycle',
        'horse',
        'boat',
        'motorbike',
        'cat',
        'tvmonitor',
        'cow',
        'sheep',
        'aeroplane',
        'train',
        'diningtable',
        'bus']

"""

# Create and write to the file
with open(filename, "w") as file:
    file.write(content)

print(f"File '{filename}' created and saved successfully!")


File '/kaggle/working/yolov5/data.yaml' created and saved successfully!


In [50]:
!cat data.yaml


train:  data_images/train
val: data_images/test
nc: 19
names: ['person',
        'car',
        'chair',
        'bottle',
        'pottedplant',
        'bird',
        'dog',
        'sofa',
        'bicycle',
        'horse',
        'boat',
        'motorbike',
        'cat',
        'tvmonitor',
        'cow',
        'sheep',
        'aeroplane',
        'train',
        'diningtable',
        'bus']



In [51]:
import os
os.chdir("/kaggle/working/yolov5")



In [52]:
!python3 train.py --data data.yaml --batch-size 8 --name Model --epochs 35


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 976.9/976.9 kB 19.4 MB/s eta 0:00:0000:01
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-03-31 14:47:12.073312: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-31 14:47:12.293223: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-31 14:47:12.3

In [53]:
!python export.py --weights runs/train/Model/weights/best.pt --include torchscript onnx

export: data=data/coco128.yaml, weights=['runs/train/Model/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, per_tensor=False, dynamic=False, cache=, simplify=False, mlmodel=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['torchscript', 'onnx']
YOLOv5 🚀 v7.0-411-gf4d8a84c Python-3.10.12 torch-2.5.1+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7064065 parameters, 0 gradients, 15.9 GFLOPs

PyTorch: starting from runs/train/Model/weights/best.pt with output shape (1, 25200, 25) (13.7 MB)

TorchScript: starting export with torch 2.5.1+cu121...
TorchScript: export success ✅ 1.9s, saved as runs/train/Model/weights/best.torchscript (27.4 MB)

ONNX: starting export with onnx 1.17.0...
ONNX: export success ✅ 0.8s, saved as runs/train/Model/weights/best.onnx (27.4 MB)

Export complete (3.3s)
Results sav

In [55]:
!zip -r workspace.zip /kaggle/working/yolov5/runs


  adding: kaggle/working/yolov5/runs/ (stored 0%)
  adding: kaggle/working/yolov5/runs/train/ (stored 0%)
  adding: kaggle/working/yolov5/runs/train/Model/ (stored 0%)
  adding: kaggle/working/yolov5/runs/train/Model/labels.jpg (deflated 15%)
  adding: kaggle/working/yolov5/runs/train/Model/val_batch2_labels.jpg (deflated 8%)
  adding: kaggle/working/yolov5/runs/train/Model/confusion_matrix.png (deflated 20%)
  adding: kaggle/working/yolov5/runs/train/Model/val_batch2_pred.jpg (deflated 8%)
  adding: kaggle/working/yolov5/runs/train/Model/val_batch0_pred.jpg (deflated 8%)
  adding: kaggle/working/yolov5/runs/train/Model/train_batch1.jpg (deflated 4%)
  adding: kaggle/working/yolov5/runs/train/Model/labels_correlogram.jpg (deflated 33%)
  adding: kaggle/working/yolov5/runs/train/Model/results.csv (deflated 82%)
  adding: kaggle/working/yolov5/runs/train/Model/val_batch1_labels.jpg (deflated 7%)
  adding: kaggle/working/yolov5/runs/train/Model/F1_curve.png (deflated 15%)
  adding: kaggle